In [1]:
import os
import librosa
import librosa.display
import pandas as pd
import numpy as np
import random

import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
import shutil

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, confusion_matrix

In [2]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Caminho para os dados no Google Drive
file_path_real = '/content/drive/MyDrive/amostra_FOR/real'
file_path_fake = '/content/drive/MyDrive/amostra_FOR/fake'

In [4]:
# Listar os arquivos e subpastas
print(os.listdir(file_path_real))
print(os.listdir(file_path_fake))

['file1.wav', 'file10.wav', 'file100.wav', 'file101.wav', 'file109.wav', 'file11.wav', 'file111.wav', 'file114.wav', 'file12.wav', 'file13.wav', 'file14.wav', 'file15.wav', 'file16.wav', 'file17.wav', 'file18.wav', 'file19.wav', 'file200.wav', 'file20.wav', 'file2.wav', 'file204.wav', 'file202.wav', 'file213.wav', 'file21.wav', 'file22.wav', 'file2245.wav', 'file2243.wav', 'file2248.wav', 'file2239.wav', 'file2246.wav', 'file2247.wav', 'file2227.wav', 'file2228.wav', 'file2229.wav', 'file2237.wav', 'file2240.wav', 'file2236.wav', 'file2231.wav', 'file2242.wav', 'file2244.wav', 'file2232.wav', 'file2230.wav', 'file2238.wav', 'file2241.wav', 'file2235.wav', 'file2234.wav', 'file2249.wav', 'file2233.wav', 'file2262.wav', 'file2263.wav', 'file2250.wav', 'file2256.wav', 'file24.wav', 'file2259.wav', 'file2260.wav', 'file2254.wav', 'file2264.wav', 'file2255.wav', 'file2261.wav', 'file2251.wav', 'file23.wav', 'file2258.wav', 'file2257.wav', 'file2253.wav', 'file2252.wav', 'file25.wav', 'file2

In [5]:
# Função para processar os áudios em uma pasta
def process_audio_files(folder_path):

    for audio_file in os.listdir(folder_path):
        audio_path = os.path.join(folder_path, audio_file)

        # Carregar o áudio
        y, sr = librosa.load(audio_path, sr=None)

        # Mostrar informações do áudio
        print(f"Arquivo: {audio_file}, Duração: {len(y) / sr:.2f}s, Taxa de amostragem: {sr}Hz")

        # Plotar waveform
        plt.figure(figsize=(10, 4))
        librosa.display.waveshow(y, sr=sr)
        plt.title(f"Waveform de {audio_file}")
        plt.xlabel("Tempo (s)")
        plt.ylabel("Amplitude")
        plt.show()

In [6]:
# Processar áudios Real
#process_audio_files(file_path_real)

In [7]:
# Processar áudios Fake
#process_audio_files(file_path_fake)

In [8]:
# Função para extrair MFCCs de um arquivo
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)  # Média dos coeficientes

# Carregar dados e rótulos
data = []
labels = []

# Processar Fake
for file_name in os.listdir(file_path_fake):
    file_path = os.path.join(file_path_fake, file_name)
    features = extract_features(file_path)
    data.append(features)
    labels.append(0)  # Rótulo 0 para Fake

# Processar Real
for file_name in os.listdir(file_path_real):
    file_path = os.path.join(file_path_real, file_name)
    features = extract_features(file_path)
    data.append(features)
    labels.append(1)  # Rótulo 1 para Real

In [9]:
# Converter para DataFrame
df = pd.DataFrame(data)
df['label'] = labels

In [10]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
0,-310.484985,79.589180,12.035122,21.775892,7.463613,-4.327323,1.702638,5.056441,-11.605008,-8.825641,-12.136454,-8.166679,-7.886450,0
1,-292.760193,85.521347,-18.103905,14.233822,-1.934288,-8.300287,-15.360986,-14.333099,-0.524541,-12.453050,-2.744914,-14.212177,0.993812,0
2,-266.020477,106.608009,-11.326435,17.970966,-11.344944,1.250991,-5.587977,-6.984388,-5.437147,-10.668830,6.232262,-13.461805,6.319636,0
3,-285.364471,107.933838,0.812730,12.481999,3.702962,-0.148384,-0.322461,-4.362314,-5.519533,-5.403917,-15.683066,-12.309880,-2.730593,0
4,-268.046997,93.126953,-8.684072,9.837867,7.469717,0.509814,-1.389165,-1.878421,-3.965281,-6.611226,-11.360195,-10.123342,-4.538263,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,-308.178833,79.818802,3.456124,16.118801,-18.006796,-22.130159,-21.046402,-15.056511,-13.789603,-3.872344,-19.306547,-6.663213,-16.349468,1
704,-347.042694,193.137390,-30.787479,9.523950,25.206856,13.511885,-9.119321,-3.360011,9.663671,-18.556343,-0.729798,10.197483,-2.653413,1
705,-268.225555,94.514771,-20.598610,10.906498,-8.953009,-23.462545,-19.558641,-24.115459,-19.402403,-0.642533,-16.554873,-8.804280,-15.076769,1
706,-301.363037,177.069092,-37.498245,5.008915,8.963133,-2.310470,-10.783433,-15.959943,-5.289495,-14.481618,-3.625240,-2.413878,-10.062649,1


In [11]:
# Separar características (X) e rótulos (y)
X = df.drop(columns=['label'])
y = df['label']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Treinando um modelo de regressão linear
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = regressor.predict(X_test)

In [13]:
# Calculando métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Erro Absoluto Médio (MAE): {mae:.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2f}")

Erro Absoluto Médio (MAE): 0.31
Coeficiente de Determinação (R²): 0.37


In [ ]:
# Criar a matriz de confusão
cm = confusion_matrix(y_true, y_pred)

# Plotar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'])
plt.xlabel('Valores Previstos')
plt.ylabel('Valores Reais')
plt.title('Matriz de Confusão')
plt.show()

In [14]:
def cut_and_resize(audio, sr, target_length, num_versions=3):

    target_samples = int(target_length * sr)  # Convertendo segundos para amostras
    versions = []

    for _ in range(num_versions):
        # Se o áudio for maior, corte em uma posição aleatória
        if len(audio) > target_samples:
            start = np.random.randint(0, len(audio) - target_samples)
            resized_audio = audio[start:start + target_samples]
        else:
            # Se for menor, preencha com zeros
            resized_audio = np.pad(audio, (0, target_samples - len(audio)), 'constant')

        versions.append(resized_audio)

    return versions

def augment_and_create_combined_dir(input_dir, combined_dir, target_length=5, num_versions=3):

    # Criar diretório consolidado
    os.makedirs(combined_dir, exist_ok=True)

    # Iterar sobre os arquivos no diretório de entrada
    for file_name in os.listdir(input_dir):
        if file_name.endswith(".wav"):
            file_path = os.path.join(input_dir, file_name)

            # Copiar o arquivo original para o diretório consolidado
            shutil.copy(file_path, os.path.join(combined_dir, file_name))

            # Carrega o áudio
            audio, sr = librosa.load(file_path, sr=None)

            # Aplicar corte e redimensionamento
            augmented_versions = cut_and_resize(audio, sr, target_length, num_versions=num_versions)

            # Salvar áudios aumentados no diretório consolidado
            base_name = os.path.basename(file_name).split('.')[0]
            for i, augmented_audio in enumerate(augmented_versions):
                augmented_file_name = f"{base_name}_augmented_{i + 1}.wav"
                output_file = os.path.join(combined_dir, augmented_file_name)
                write(output_file, sr, augmented_audio)
                print(f"Áudio misturado salvo em: {output_file}")

# Caminho para os dados no Google Drive
file_path_real = '/content/drive/MyDrive/amostra_FOR/real'
file_path_fake = '/content/drive/MyDrive/amostra_FOR/fake'

# Diretórios para os dados combinados
combined_dir_real = '/content/drive/MyDrive/amostra_FOR_combined/real'
combined_dir_fake = '/content/drive/MyDrive/amostra_FOR_combined/fake'

# Processar ambas as categorias e criar diretórios consolidados com múltiplas versões
augment_and_create_combined_dir(file_path_real, combined_dir_real, target_length=5, num_versions=3)
augment_and_create_combined_dir(file_path_fake, combined_dir_fake, target_length=5, num_versions=3)

Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file1_augmented_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file1_augmented_2.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file1_augmented_3.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file10_augmented_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file10_augmented_2.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file10_augmented_3.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file100_augmented_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file100_augmented_2.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file100_augmented_3.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file101_augmented_1.wav
Áudio misturado sal

In [15]:
# Função para extrair MFCCs de um arquivo
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)  # Média dos coeficientes

# Diretórios de saída distintos para áudios 'real' e 'fake'
combined_dir_real = '/content/drive/MyDrive/amostra_FOR_combined/real'
combined_dir_fake = '/content/drive/MyDrive/amostra_FOR_combined/fake'

# Listas para armazenar os dados e rótulos
data = []
labels = []

# Processar Fake
for file_name in os.listdir(combined_dir_fake):
    if file_name.endswith(".wav"):  # Verificar se é um arquivo .wav
        file_path = os.path.join(combined_dir_fake, file_name)
        features = extract_features(file_path)
        data.append(features)
        labels.append(0)  # Rótulo 0 para Fake

# Processar Real
for file_name in os.listdir(combined_dir_real):
    if file_name.endswith(".wav"):  # Verificar se é um arquivo .wav
        file_path = os.path.join(combined_dir_real, file_name)
        features = extract_features(file_path)
        data.append(features)
        labels.append(1)  # Rótulo 1 para Real

In [16]:
# Converter para DataFrame
df_cut_and_resize = pd.DataFrame(data)
df_cut_and_resize['label'] = labels

In [17]:
df_cut_and_resize

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
0,-310.484985,79.589180,12.035122,21.775892,7.463613,-4.327323,1.702638,5.056441,-11.605008,-8.825641,-12.136454,-8.166679,-7.886450,0
1,-472.288971,31.496994,4.762836,8.617694,2.953685,-1.712515,0.673810,2.001060,-4.592621,-3.492700,-4.802937,-3.231920,-3.121021,0
2,-472.288971,31.496994,4.762836,8.617694,2.953685,-1.712515,0.673810,2.001060,-4.592621,-3.492700,-4.802937,-3.231920,-3.121021,0
3,-472.288971,31.496994,4.762836,8.617694,2.953685,-1.712515,0.673810,2.001060,-4.592621,-3.492700,-4.802937,-3.231920,-3.121021,0
4,-292.760193,85.521347,-18.103905,14.233822,-1.934288,-8.300287,-15.360986,-14.333099,-0.524541,-12.453050,-2.744914,-14.212177,0.993812,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215,-373.319519,132.753006,-28.106714,3.802946,6.691942,-1.724923,-8.047286,-11.982522,-3.965743,-10.869507,-2.749518,-1.817298,-7.560544,1
2216,-308.821716,57.043686,3.870187,30.615040,-13.108034,-13.102796,-21.079472,-21.996910,-15.296227,-3.665011,-15.141479,1.185465,-17.169489,1
2217,-528.484009,19.380270,1.424431,10.445410,-4.436643,-4.370504,-7.140476,-7.444872,-5.124530,-1.271869,-5.072576,0.414355,-5.821797,1
2218,-528.484009,19.380270,1.424431,10.445410,-4.436643,-4.370504,-7.140476,-7.444872,-5.124530,-1.271869,-5.072576,0.414355,-5.821797,1


In [18]:
# Separar características (X) e rótulos (y)
X = df_cut_and_resize.drop(columns=['label'])
y = df_cut_and_resize['label']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Treinando um modelo de regressão linear
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = regressor.predict(X_test)

In [20]:
# Calculando métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Erro Absoluto Médio (MAE): {mae:.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2f}")

Erro Absoluto Médio (MAE): 0.23
Coeficiente de Determinação (R²): 0.56


In [ ]:
# Criar a matriz de confusão
cm = confusion_matrix(y_true, y_pred)

# Plotar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'])
plt.xlabel('Valores Previstos')
plt.ylabel('Valores Reais')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
def mix_with_other_audio(audio, sr, input_dir, num_mixes=2):
    """
    Mistura o áudio fornecido com outros áudios do diretório de entrada.
    """
    mixed_versions = []
    audio_files = [f for f in os.listdir(input_dir) if f.endswith(".wav")]

    for _ in range(num_mixes):
        try:
            # Escolher um arquivo aleatório para mistura
            other_file = random.choice(audio_files)
            other_path = os.path.join(input_dir, other_file)

            # Carregar o áudio a ser misturado
            other_audio, _ = librosa.load(other_path, sr=sr)

            # Ajustar o tamanho do áudio misturado
            min_length = min(len(audio), len(other_audio))
            trimmed_audio = audio[:min_length]
            trimmed_other_audio = other_audio[:min_length]

            # Misturar os áudios com pesos aleatórios
            weight_original = random.uniform(0.5, 0.9)  # Peso para o áudio original
            weight_other = 1.0 - weight_original  # Peso complementar para o outro áudio
            mixed_audio = (weight_original * trimmed_audio) + (weight_other * trimmed_other_audio)

            # Normalizar e garantir que o áudio misturado esteja no intervalo [-1, 1]
            mixed_audio = mixed_audio / np.max(np.abs(mixed_audio))  # Normalizar
            mixed_audio = np.clip(mixed_audio, -1.0, 1.0)  # Garantir que os valores fiquem dentro do intervalo

            mixed_versions.append(mixed_audio)
        except Exception as e:
            print(f"Erro ao processar o arquivo {other_file}: {e}")

    return mixed_versions

def generate_mixed_data(input_dir, output_dir_fake, output_dir_real, num_mixes=2):
    """
    Gera versões misturadas de áudios e salva em subpastas 'fake' e 'real'.
    """
    # Criar as pastas 'fake' e 'real' para salvar os áudios misturados
    os.makedirs(output_dir_fake, exist_ok=True)
    os.makedirs(output_dir_real, exist_ok=True)
    print(f"Diretórios '{output_dir_fake}' e '{output_dir_real}' criados com sucesso.")

    for file_name in os.listdir(input_dir):
        if file_name.endswith(".wav"):
            try:
                file_path = os.path.join(input_dir, file_name)

                # Carregar o áudio original
                audio, sr = librosa.load(file_path, sr=None)

                # Aplicar mistura com outros áudios do mesmo diretório
                mixed_versions = mix_with_other_audio(audio, sr, input_dir, num_mixes=num_mixes)

                # Salvar versões misturadas nos diretórios 'fake' ou 'real'
                base_name = os.path.basename(file_name).split('.')[0]
                for i, mixed_audio in enumerate(mixed_versions):
                    mixed_file_name = f"{base_name}_mixed_{i + 1}.wav"
                    if 'fake' in file_name.lower():
                        output_file = os.path.join(output_dir_fake, mixed_file_name)
                    else:
                        output_file = os.path.join(output_dir_real, mixed_file_name)

                    write(output_file, sr, (mixed_audio * 32767).astype(np.int16))  # Converter para int16
                    print(f"Áudio misturado salvo em: {output_file}")
            except Exception as e:
                print(f"Erro ao processar o arquivo {file_name}: {e}")

# Exemplos de diretórios de entrada e saída
redirected_real = '/content/drive/MyDrive/amostra_FOR_combined/real'
redirected_fake = '/content/drive/MyDrive/amostra_FOR_combined/fake'

# Diretórios de saída distintos para áudios 'real' e 'fake'
output_dir_real_mixed = '/content/drive/MyDrive/amostra_FOR_combined_mixed/real'
output_dir_fake_mixed = '/content/drive/MyDrive/amostra_FOR_combined_mixed/fake'

# Gerar dados misturados a partir dos áudios redirecionados
generate_mixed_data(redirected_real, output_dir_fake_mixed, output_dir_real_mixed, num_mixes=2)
generate_mixed_data(redirected_fake, output_dir_fake_mixed, output_dir_real_mixed, num_mixes=2)

Diretórios '/content/drive/MyDrive/amostra_FOR_combined_mixed/fake' e '/content/drive/MyDrive/amostra_FOR_combined_mixed/real' criados com sucesso.
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file1_mixed_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file1_mixed_2.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file1_augmented_1_mixed_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file1_augmented_1_mixed_2.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file1_augmented_2_mixed_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file1_augmented_2_mixed_2.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file1_augmented_3_mixed_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file1_augmented_3_

In [ ]:
# Função para extrair MFCCs de um arquivo
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)  # Média dos coeficientes

# Diretórios de saída distintos para áudios 'real' e 'fake'
output_dir_real_mixed = '/content/drive/MyDrive/amostra_FOR_combined_mixed/real'
output_dir_fake_mixed = '/content/drive/MyDrive/amostra_FOR_combined_mixed/fake'

# Listas para armazenar os dados e rótulos
data = []
labels = []

# Processar Fake
for file_name in os.listdir(output_dir_fake_mixed):
    if file_name.endswith(".wav"):  # Verificar se é um arquivo .wav
        file_path = os.path.join(output_dir_fake_mixed, file_name)
        features = extract_features(file_path)
        data.append(features)
        labels.append(0)  # Rótulo 0 para Fake

# Processar Real
for file_name in os.listdir(output_dir_real_mixed):
    if file_name.endswith(".wav"):  # Verificar se é um arquivo .wav
        file_path = os.path.join(output_dir_real_mixed, file_name)
        features = extract_features(file_path)
        data.append(features)
        labels.append(1)  # Rótulo 1 para Real

In [ ]:
# Converter para DataFrame
df_mix_with_other_audio = pd.DataFrame(data)
df_mix_with_other_audio['label'] = labels

In [ ]:
df_mix_with_other_audio

In [ ]:
# Separar características (X) e rótulos (y)
X = df_mix_with_other_audio.drop(columns=['label'])
y = df_mix_with_other_audio['label']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Treinando um modelo de regressão linear
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = regressor.predict(X_test)

In [ ]:
# Calculando métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Erro Absoluto Médio (MAE): {mae:.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2f}")

In [ ]:
# Criar a matriz de confusão
cm = confusion_matrix(y_true, y_pred)

# Plotar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'])
plt.xlabel('Valores Previstos')
plt.ylabel('Valores Reais')
plt.title('Matriz de Confusão')
plt.show()